In [1]:
import requests
import pandas as pd

Using the following base table for queries: `https://phl.carto.com/api/v2/sql?q=SELECT * FROM violations`

In [2]:
earliest_violation = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=SELECT * FROM violations v order by casecreateddate asc limit 1
    """
).json()["rows"][0]["casecreateddate"]
latest_violation = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=SELECT * FROM violations v where casecreateddate is not null order by casecreateddate desc limit 1
    """
).json()["rows"][0]["casecreateddate"]
n_violations = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=SELECT count(*) FROM violations v
"""
).json()["rows"][0]["count"]
print(
    f"There have been {n_violations} violations between {earliest_violation} and {latest_violation}"
)

There have been 1551040 violations between 2007-01-01T19:00:41Z and 2021-10-20T18:36:26Z


In [3]:
print("Example Violation")
requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=SELECT * FROM violations v order by violationdate desc limit 1
    """
).json()["rows"][0]

Example Violation


{'cartodb_id': 171358,
 'the_geom': '0101000020E6100000750AF29477CA52C07E7B3F8601FD4340',
 'the_geom_webmercator': '0101000020110F00003452E2FD0FEB5FC1B542E2EF938C5241',
 'objectid': 177127,
 'addressobjectid': '15432928',
 'parcel_id_num': '459235',
 'casenumber': 'CF-2021-102790',
 'casecreateddate': '2021-10-20T08:45:10Z',
 'casecompleteddate': None,
 'casetype': 'NOTICE OF VIOLATION',
 'casestatus': 'IN VIOLATION',
 'caseresponsibility': 'CSU INVESTIGATOR',
 'caseprioritydesc': 'UNSAFE',
 'violationnumber': 'VI-2021-074178',
 'violationdate': '2021-10-20T00:00:00Z',
 'violationcode': 'PM15-108.1',
 'violationcodetitle': 'UNSAFE STRUCTURE',
 'violationstatus': 'OPEN',
 'violationresolutiondate': None,
 'violationresolutioncode': None,
 'mostrecentinvestigation': '2021-10-20T08:56:45Z',
 'opa_account_num': '471260900',
 'address': '1500 N 17TH ST',
 'unit_type': None,
 'unit_num': None,
 'zip': '19121-4209',
 'censustract': '147',
 'opa_owner': 'NALLEY JOHNNIE CLARKE',
 'systemofrecor

In [9]:
print("Distinct statuses")
requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=SELECT violationstatus, count(*) FROM violations v group by violationstatus
    """
).json()["rows"]

Distinct statuses


[{'violationstatus': 'CLOSED', 'count': 10016},
 {'violationstatus': 'CLOSEDCASE', 'count': 124309},
 {'violationstatus': 'CMPLY', 'count': 8666},
 {'violationstatus': 'COMPEXCP', 'count': 29606},
 {'violationstatus': 'COMPLIED', 'count': 1229617},
 {'violationstatus': 'CVN ISSUED', 'count': 1293},
 {'violationstatus': 'DEMOLISH', 'count': 21747},
 {'violationstatus': 'ERROR', 'count': 17029},
 {'violationstatus': 'OPEN', 'count': 83853},
 {'violationstatus': 'RESOLVE', 'count': 7144},
 {'violationstatus': 'STOP WORK', 'count': 139},
 {'violationstatus': 'SVN ISSUED', 'count': 5},
 {'violationstatus': 'WARNING ISSUED', 'count': 20},
 {'violationstatus': None, 'count': 17596}]

In [4]:
violation_counts = requests.get(
    """
https://phl.carto.com/api/v2/sql?q=SELECT caseprioritydesc, violationcode, violationcodetitle, count(cartodb_id) 
FROM violations
group by violationcode, violationcodetitle, caseprioritydesc
"""
)

violation_counts_df = pd.DataFrame(violation_counts.json()["rows"]).sort_values(
    "count", ascending=False
)

print(
    violation_counts_df.groupby("caseprioritydesc")["count"]
    .sum()
    .sort_values(ascending=False)
)
print()

for caseprioritydesc in violation_counts_df["caseprioritydesc"].unique():
    if caseprioritydesc:
        sub_df = violation_counts_df[
            violation_counts_df["caseprioritydesc"] == caseprioritydesc
        ]
        print(f"{caseprioritydesc} with {sub_df['count'].sum()} entries")
        print(
            sub_df[["violationcode", "violationcodetitle", "count"]]
            .head(5)
            .reset_index()
            .drop("index", axis=1)
        )
        print()

caseprioritydesc
STANDARD                          1256128
HAZARDOUS                           96658
CONSTRUCTION SERVICES               80737
UNSAFE                              64397
IMMINENTLY DANGEROUS                25599
ACCELERATED REVIEW                     40
AIU LICENSING VIOLATION NOTICE         28
UNLAWFUL                               24
5 DAY REVIEW GROUP                      3
UNFIT                                   3
Name: count, dtype: int64

STANDARD with 1256128 entries
  violationcode              violationcodetitle   count
0         CP-01           CLIP VIOLATION NOTICE  212467
1       CP-312A                  HIGH WEEDS-CUT   80216
2    PM-302.2/4  EXT A-VACANT LOT CLEAN/MAINTAI   73461
3        CP-305  RUBBISH/GARBAGE EXTERIOR-OWNER   47189
4    PM15-302.4             EXTERIOR AREA WEEDS   33707

UNSAFE with 64397 entries
  violationcode             violationcodetitle  count
0    PM15-108.1               UNSAFE STRUCTURE  12831
1   PM15-304.1G  EXTERIOR STRUCT UN

In [6]:
n_unique_codeviolations = requests.get(
    'https://phl.carto.com/api/v2/sql?q=SELECT count(distinct("violationcode")) FROM violations'
).json()["rows"][0]["count"]
n_caseprioritydesc = requests.get(
    'https://phl.carto.com/api/v2/sql?q=SELECT count(distinct("caseprioritydesc")) FROM violations'
).json()["rows"][0]["count"]
print(
    f"There are {n_unique_codeviolations} unique code violations and {n_caseprioritydesc} priority levels"
)

There are 2312 unique code violations and 11 priority levels


In [7]:
caseprioritydesc = [
    c["caseprioritydesc"]
    for c in requests.get(
        'https://phl.carto.com/api/v2/sql?q=SELECT distinct("caseprioritydesc") FROM violations'
    ).json()["rows"]
]
print(f"Case Priority Descriptions: {caseprioritydesc}")

Case Priority Descriptions: ['UNSAFE', 'HAZARDOUS', 'CONSTRUCTION SERVICES', 'ACCELERATED REVIEW', 'UNFIT', 'STANDARD', 'HAZARD', 'UNLAWFUL', None, 'NON HAZARDOUS', 'IMMINENTLY DANGEROUS', '5 DAY REVIEW GROUP']


In [8]:
n_matching_addresses = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=SELECT count(distinct(opp.parcel_number)) FROM violations v, opa_properties_public opp where v.opa_account_num= opp.parcel_number
    and opp.location = v.address
"""
).json()["rows"][0]["count"]

n_non_matching_addresses = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=SELECT count(distinct(opp.parcel_number)) FROM violations v, opa_properties_public opp where v.opa_account_num= opp.parcel_number
    and opp.location != v.address
"""
).json()["rows"][0]["count"]

In [9]:
pct = (
    round(
        n_non_matching_addresses / (n_non_matching_addresses + n_matching_addresses), 2
    )
    * 100
)
print(
    f"There are {n_matching_addresses} properties that perfectly match the opa_properties_public.location to violations.address"
)
print(
    f"There are {n_non_matching_addresses} properties that do not perfectly match the opa_properties_public.location to violations.address, about {pct}%"
)
print("However, a spot check seems like they are in fact the same property.")

There are 207687 properties that perfectly match the opa_properties_public.location to violations.address
There are 4399 properties that do not perfectly match the opa_properties_public.location to violations.address, about 2.0%
However, a spot check seems like they are in fact the same property.


### Other notes about this data

### Illegal Rentals

- [Violation 9-3902](https://codelibrary.amlegal.com/codes/philadelphia/latest/philadelphia_pa/0-0-0-197435)
- [Prediction Models](https://pennmusa.github.io/MUSA_801.io/project_7/index.html)

In [33]:
illegal_rental_count = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=SELECT count(v.*)
    FROM violations v, opa_properties_public opp where v.opa_account_num= opp.parcel_number
    and opp.location = v.address 
    and v.violationcode like '%9-3902%'
"""
).json()["rows"][0]["count"]
print(
    f"There are {illegal_rental_count} violations for illegal rentals in the dataset."
)

illegal_rental_count_grouped = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=SELECT opp.category_code_description, count(v.*)
    FROM violations v, opa_properties_public opp where v.opa_account_num= opp.parcel_number
    and opp.location = v.address 
    and v.violationcode like '%9-3902%'
    group by opp.category_code_description
"""
).json()["rows"]
pd.DataFrame(illegal_rental_count_grouped)

There are 10197 violations for illegal rentals in the dataset.


,category_code_description,count
0,Commercial,51
1,Industrial,12
2,Mixed Use,980
3,Multi Family,2874
4,Single Family,6232
5,Vacant Land,48


In [ ]:
illegal_rental_results = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=SELECT v.*
    FROM violations v, opa_properties_public opp where v.opa_account_num= opp.parcel_number
    and opp.location = v.address 
    and v.violationcode like '%9-3902%'
    limit 10
"""
).json()["rows"]
pd.DataFrame(illegal_rental_results).head()